In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as func

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
# two 3*3 convolution residual block
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.residual = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )
 
    def forward(self, x):
        out = self.residual(x)
        out += self.shortcut(x)
        out = func.relu(out)
        return out

# Residual network
class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            # small initial convolution
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        #fc with dropout
        self.fc = nn.Linear(512, 512)
        self.drop = nn.Dropout2d(0.5)
        self.fc2 = nn.Linear(512, num_classes)
 
    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)
 
    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = func.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        out = self.drop(out)
        out = self.fc2(out)
        return out


def ResNet18():
    return ResNet(ResidualBlock)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device: ', device)
if device = "cpu":
    print("Please delete all .cuda() and .cpu() while using a cpu.")

In [ ]:
model = ResNet18().cuda()
pretrained_model = 'ResNet18.path'
# change map_location to cpu if needed
model.load_state_dict(torch.load(pretrained_model, map_location='cuda'))
model.eval()

In [ ]:
# Self-defined testset
class TestSet(Dataset):
    def __init__(self, data,transform=None):
        self.data = data
        self.transform = transform
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.transform(self.data[idx])
# same normalization as training set
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Prepare data
y_test = np.load("test_data/y_test.npy").astype(np.uint8)
dataset = TestSet(y_test, transform=transform_test)
testLoader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

In [ ]:
# Predict
category = []
with torch.no_grad():
    for data in testLoader:
        image = data.type('torch.FloatTensor').cuda()
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)
        category.append(predicted.cpu().numpy()[0])

In [ ]:
dataframe = pd.DataFrame({'Index':np.arange(0,12000),'Category':category})
dataframe.to_csv("result.csv",index=False,sep=',')